# Home Credit Project - Data Audit

## Header
Version 0.1

Date: 20/06/2024

Author: Dale Chancellor

### Code Purpose
This code perfroms simple data quality checks on the datasets to be used for the Home Credit Model Build project on Kaggle (https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability/overview)


### 0. Setup

In [137]:
# Import libraries
import pandas as pd
import numpy as np

In [138]:
# Define data location
train_path = str('../Data/train')
test_path = str('../Data/test')

In [139]:
# Threshold to classify as continuous
cont_threshold = 20

# List of datasets to be analysed
target_csvs = ['train_tax_registry_a_1','train_tax_registry_b_1','train_tax_registry_c_1']

# continuous variable classifier
classify_cont = lambda x: True if x > cont_threshold else False

### 1. Functions

Column Stats: 
,unique values
,data type
,table name

Overall Stats
number of.. categorical, continuous, numeric, character
,Total rows
,Missings (any)
,Missing ID?
,ID variables

NB: Dates may be read as categorical if stored as text, some INT categories may be ID variables

In [142]:
def variable_summary(input_df):
    ## Column stats
    # Count number of unique values
    input_uniqvals = pd.DataFrame(input_df.nunique(axis=0),columns=["Unique Values"])

    # Get the data types of each column
    input_dtypes = pd.DataFrame(input_df.dtypes, columns=['Data Type'])

    # Merge to create one dataframe
    var_summary = pd.merge(left=input_uniqvals,right=input_dtypes,left_index=True,right_index=True)
    
    # Get total number of missings per column
    var_summary['Missings'] = input_df.isna().sum()

    # Get continuous and categorical variables
    var_summary['Variable Type'] = np.where((var_summary['Unique Values'].apply(classify_cont)) 
                                            & (var_summary['Data Type'] != 'object'), 'Continuous', 'Categorical')

    # Table name
    var_summary['Table Name'] = table_name
    # Get total number of rows
    var_summary['Total Rows'] = input.shape[0]

    var_summary.index.name='Variable Name'

    return var_summary



In [143]:
def table_summary(input_df):
    # Define index for tables
    overall_index = pd.Index(data=[table_name],name='Table Name')

    # Create blank DataFrame
    overall_summary = pd.DataFrame(index=overall_index)

    # Get total number of variables
    overall_summary['Total Vars'] = input_df.shape[1]
    # Total number of rows
    overall_summary['Total Rows'] = input_df.shape[0]

    # Number of categorical and continuous vars
    cat_con = var_summary.groupby('Variable Type').count()['Total Rows'].to_frame().T
    cat_con.set_index(overall_index, inplace=True)

    # Join Categorical and Continuous to overall table
    overall_summary = pd.merge(left=overall_summary, right=cat_con, left_index = True, right_index=True)

    # Count total number of rows with any missing
    overall_summary['Any Missing'] = np.count_nonzero(input_df.isna().sum(axis=1))

    return overall_summary


### 2. Function Execution

In [144]:
for table_name in target_csvs:
    # Import table
    input = pd.read_csv(train_path+'/'+table_name+'.csv')

    # Variable summary
    var_summ = variable_summary(input)
    print(var_summ)

    # Overall Summary
    over_summ = table_summary(input)
    print(over_summ)

                     Unique Values Data Type  Missings Variable Type  \
Variable Name                                                          
case_id                     457934     int64         0    Continuous   
amount_4527230A              92743   float64         0    Continuous   
name_4527232M               147037    object         0   Categorical   
num_group1                      99     int64         0    Continuous   
recorddate_4527225D            397    object         0   Categorical   

                                 Table Name  Total Rows  
Variable Name                                            
case_id              train_tax_registry_a_1     3275770  
amount_4527230A      train_tax_registry_a_1     3275770  
name_4527232M        train_tax_registry_a_1     3275770  
num_group1           train_tax_registry_a_1     3275770  
recorddate_4527225D  train_tax_registry_a_1     3275770  
                        Total Vars  Total Rows  Categorical  Continuous  \
Table Name    